In [83]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [4]:
path = "data/cleaned_data.csv"

In [18]:
df = pd.read_csv(path)

In [19]:
data = df.copy()

In [20]:
data.shape

(15215, 35)

In [21]:
data.columns

Index(['permit number', 'permit type', 'permit type definition',
       'permit creation date', 'block', 'lot', 'street number', 'street name',
       'street suffix', 'description', 'current status', 'current status date',
       'filed date', 'issued date', 'completed date',
       'first construction document date', 'number of existing stories',
       'number of proposed stories', 'permit expiration date',
       'estimated cost', 'revised cost', 'existing use', 'existing units',
       'proposed use', 'proposed units', 'plansets',
       'existing construction type', 'existing construction type description',
       'proposed construction type', 'proposed construction type description',
       'supervisor district', 'neighborhoods - analysis boundaries', 'zipcode',
       'location', 'record id'],
      dtype='object')

In [22]:
data.head()

,permit number,permit type,permit type definition,permit creation date,block,lot,street number,street name,street suffix,description,current status,current status date,filed date,issued date,completed date,first construction document date,number of existing stories,number of proposed stories,permit expiration date,estimated cost,revised cost,existing use,existing units,proposed use,proposed units,plansets,existing construction type,existing construction type description,proposed construction type,proposed construction type description,supervisor district,neighborhoods - analysis boundaries,zipcode,location,record id
0,201505000000.0,4,sign - erect,2015-05-06,326,23,140,Ellis,St,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,2015-05-06,2015-11-09,2013-03-21,2015-11-09,6.0,6.0,2016-11-03,4000.0,4000.0,tourist hotel/motel,143.0,museum,160.0,2.0,3.0,constr type 3,1.0,constr type 1,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1.380610e+12
1,201604000000.0,4,sign - erect,2016-04-19,306,7,440,Geary,St,remove (e) awning and associated signs.,issued,08/03/2017,2016-04-19,2017-08-03,2013-03-21,2017-08-03,7.0,7.0,2017-12-03,1.0,500.0,tourist hotel/motel,36.0,misc group residns.,36.0,2.0,3.0,constr type 3,2.0,constr type 2,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1.420160e+12
2,201605000000.0,3,additions alterations or repairs,2016-05-27,595,203,1647,Pacific,Av,installation of separating wall,withdrawn,09/26/2017,2016-05-27,2013-06-03,2013-03-21,2013-06-03,6.0,6.0,2014-05-29,20000.0,25000.0,retail sales,39.0,retail sales,39.0,2.0,1.0,constr type 1,1.0,constr type 1,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1.424860e+12
3,201611000000.0,8,otc alterations permit,2016-11-07,156,11,1230,Pacific,Av,repair dryrot & stucco at front of bldg.,complete,07/24/2017,2016-11-07,2017-07-18,2017-07-24,2017-07-18,2.0,2.0,2018-07-13,2000.0,2000.0,1 family dwelling,1.0,1 family dwelling,1.0,2.0,5.0,wood frame (5),5.0,wood frame (5),3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1.443570e+12
4,201611000000.0,6,demolitions,2016-11-28,342,1,950,Market,St,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,2016-11-28,2017-12-01,2013-03-21,2017-11-20,3.0,3.0,2018-12-01,100000.0,100000.0,retail sales,5.0,workshop commercial,8.0,2.0,3.0,constr type 3,1.0,constr type 1,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",1.445480e+11


In [23]:
data.select_dtypes(include=['float', 'int']).corr()

,permit type,street number,number of existing stories,number of proposed stories,estimated cost,revised cost,existing units,proposed units,plansets,existing construction type,proposed construction type,supervisor district,zipcode,record id
permit type,1.000000,-0.016908,0.022323,0.018089,-0.108060,-0.107840,-0.007071,-0.016602,-0.245464,0.067909,0.067179,-0.000477,0.014770,0.014865
street number,-0.016908,1.000000,-0.175092,-0.175230,-0.017012,-0.017333,-0.051983,-0.054152,-0.039319,0.163335,0.168474,-0.126037,0.114524,-0.057987
number of existing stories,0.022323,-0.175092,1.000000,0.998669,0.081559,0.067786,0.322734,0.332505,0.208006,-0.558950,-0.577779,-0.133345,-0.277345,-0.094342
number of proposed stories,0.018089,-0.175230,0.998669,1.000000,0.092160,0.076694,0.322606,0.333113,0.210115,-0.558399,-0.579647,-0.131927,-0.276871,-0.093729
estimated cost,-0.108060,-0.017012,0.081559,0.092160,1.000000,0.869941,0.029702,0.036769,0.032138,-0.036240,-0.049904,0.005449,-0.020189,0.006076
revised cost,-0.107840,-0.017333,0.067786,0.076694,0.869941,1.000000,0.025463,0.034102,0.030057,-0.030625,-0.047600,0.010443,-0.013260,0.007679
existing units,-0.007071,-0.051983,0.322734,0.322606,0.029702,0.025463,1.000000,0.990318,0.024635,-0.224122,-0.233958,-0.050783,-0.109920,-0.010938
proposed units,-0.016602,-0.054152,0.332505,0.333113,0.036769,0.034102,0.990318,1.000000,0.028369,-0.229500,-0.242780,-0.048645,-0.114787,-0.008779
plansets,-0.245464,-0.039319,0.208006,0.210115,0.032138,0.030057,0.024635,0.028369,1.000000,-0.331010,-0.317295,-0.046011,-0.135674,-0.042171
existing construction type,0.067909,0.163335,-0.558950,-0.558399,-0.036240,-0.030625,-0.224122,-0.229500,-0.331010,1.000000,0.926753,0.137015,0.309988,0.078448


In [44]:
data["permit creation date"] = pd.to_datetime(data["permit creation date"])
data["filed date"] = pd.to_datetime(data["filed date"])
data["first construction document date"] = pd.to_datetime(data["first construction document date"])
data["issued date"] = pd.to_datetime(data["issued date"])
data["permit expiration date"] = pd.to_datetime(data["permit expiration date"])
data["permit creation date"] = pd.to_datetime(data["permit creation date"])


In [62]:
data["if_permit_created_on_file_date"] = data["filed date"] == data["permit creation date"]
data["if_permit_created_on_file_date"].value_counts()

if_permit_created_on_file_date
True     15193
False       22
Name: count, dtype: int64

In [61]:
# if current_status is complete
data["if_completed"] = data["current status"] == 'complete'
data["if_completed"].value_counts()

if_completed
True     9026
False    6189
Name: count, dtype: int64

In [60]:
# if first construction document date on issue date
data["if_first_construc_is_issue_date"] = data["first construction document date"] == data["issued date"]
data["if_first_construc_is_issue_date"].value_counts()

if_first_construc_is_issue_date
True     14957
False      258
Name: count, dtype: int64

In [59]:
data["diff_permit_date"] = data["permit expiration date"] - data["permit creation date"]
data["diff_permit_date"].value_counts()

diff_permit_date
360 days     6874
361 days      637
363 days      279
364 days      244
365 days      219
             ... 
1275 days       1
2391 days       1
1374 days       1
1417 days       1
935 days        1
Name: count, Length: 811, dtype: int64

In [58]:
data["expired_before_completed"] = data["completed date"] > data["permit expiration date"]
data["expired_before_completed"].value_counts()

expired_before_completed
False    14243
True       972
Name: count, dtype: int64

In [63]:
data["ratio_stories"] = data["number of existing stories"] / data["number of proposed stories"]
data["ratio_stories"].value_counts()

ratio_stories
1.000000    14393
0.666667      182
0.750000       82
0.500000       69
0.000000       47
            ...  
1.023810        1
1.055556        1
0.974359        1
0.909091        1
0.948718        1
Name: count, Length: 86, dtype: int64

In [65]:
data["existing_proposed_construction_type"] = data["existing construction type"] == data["proposed construction type"]
data["existing_proposed_construction_type"].value_counts()

existing_proposed_construction_type
True     13559
False     1656
Name: count, dtype: int64

In [67]:
data["ratio_estimated_revised"] = data["estimated cost"] / data["revised cost"]
data["ratio_estimated_revised"].value_counts()

ratio_estimated_revised
1.000000e+00    9847
inf              346
3.000000e+03     154
5.000000e-01     146
4.000000e+03     124
                ... 
1.610900e+04       1
2.037037e-01       1
4.190325e-01       1
7.659574e-01       1
8.334091e-01       1
Name: count, Length: 1360, dtype: int64

In [70]:
data["ratio_existing_proposed_units"] = data["existing units"] / data["proposed units"]
data["ratio_existing_proposed_units"].value_counts()

ratio_existing_proposed_units
1.000000    9483
0.000000     414
inf          253
0.500000      96
0.666667      90
            ... 
1.261398       1
0.990000       1
0.940000       1
1.444444       1
1.149425       1
Name: count, Length: 432, dtype: int64

In [77]:
mm = MinMaxScaler()
data["street number"] = mm.fit_transform(data[["street number"]])

In [78]:
data.head()

,permit number,permit type,permit type definition,permit creation date,block,lot,street number,street name,street suffix,description,current status,current status date,filed date,issued date,completed date,first construction document date,number of existing stories,number of proposed stories,permit expiration date,estimated cost,revised cost,existing use,existing units,proposed use,proposed units,plansets,existing construction type,existing construction type description,proposed construction type,proposed construction type description,supervisor district,neighborhoods - analysis boundaries,zipcode,location,record id,if_completed,if_first_construc_is_issue_date,diff_permit_date,expired_before_completed,rato_stories,if_permit_created_on_file_date,ratio_stories,existing_proposed_construction_type,ratio_estimated_revised,ratio_existing_proposed_units
0,201505000000.0,4,sign - erect,2015-05-06,326,23,0.016953,Ellis,St,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,2015-05-06,2015-11-09,2013-03-21,2015-11-09,6.0,6.0,2016-11-03,4000.0,4000.0,tourist hotel/motel,143.0,museum,160.0,2.0,3.0,constr type 3,1.0,constr type 1,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1.380610e+12,False,True,547 days,False,1.0,True,1.0,False,1.000,0.89375
1,201604000000.0,4,sign - erect,2016-04-19,306,7,0.053543,Geary,St,remove (e) awning and associated signs.,issued,08/03/2017,2016-04-19,2017-08-03,2013-03-21,2017-08-03,7.0,7.0,2017-12-03,1.0,500.0,tourist hotel/motel,36.0,misc group residns.,36.0,2.0,3.0,constr type 3,2.0,constr type 2,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1.420160e+12,False,True,593 days,False,1.0,True,1.0,False,0.002,1.00000
2,201605000000.0,3,additions alterations or repairs,2016-05-27,595,203,0.200756,Pacific,Av,installation of separating wall,withdrawn,09/26/2017,2016-05-27,2013-06-03,2013-03-21,2013-06-03,6.0,6.0,2014-05-29,20000.0,25000.0,retail sales,39.0,retail sales,39.0,2.0,1.0,constr type 1,1.0,constr type 1,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1.424860e+12,False,True,-729 days,False,1.0,True,1.0,True,0.800,1.00000
3,201611000000.0,8,otc alterations permit,2016-11-07,156,11,0.149896,Pacific,Av,repair dryrot & stucco at front of bldg.,complete,07/24/2017,2016-11-07,2017-07-18,2017-07-24,2017-07-18,2.0,2.0,2018-07-13,2000.0,2000.0,1 family dwelling,1.0,1 family dwelling,1.0,2.0,5.0,wood frame (5),5.0,wood frame (5),3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1.443570e+12,True,True,613 days,False,1.0,True,1.0,True,1.000,1.00000
4,201611000000.0,6,demolitions,2016-11-28,342,1,0.115746,Market,St,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,2016-11-28,2017-12-01,2013-03-21,2017-11-20,3.0,3.0,2018-12-01,100000.0,100000.0,retail sales,5.0,workshop commercial,8.0,2.0,3.0,constr type 3,1.0,constr type 1,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",1.445480e+11,False,False,733 days,False,1.0,True,1.0,False,1.000,0.62500


In [84]:
# TODO extract long and lat from long column
# TODO transform lot with minmax scaler
# TODO check for correlation
# TODO use elasticnet to select important features
# TODO add comments